# Import librerie e definizione funzioni d'utilità

In [ ]:
import functions as fn

In [ ]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
from upsetplot import generate_counts
from upsetplot import plot
from matplotlib import pyplot
from upsetplot import from_memberships
import ast

In [ ]:
dataset_name = "cmc"
if not os.path.exists("..\\Plot"):
    os.mkdir("..\\Plot")
if not os.path.exists("..\\Plot\\" + dataset_name):
    os.mkdir("..\\Plot\\" + dataset_name)
if not os.path.exists("..\\Plot\\" + dataset_name + "\\tempi"):
    os.mkdir("..\\Plot\\" + dataset_name + "\\tempi")
if not os.path.exists("..\\Plot\\" + dataset_name + "\\venn"):
    os.mkdir("..\\Plot\\" + dataset_name + "\\venn")
if not os.path.exists("..\\DatasetParsed"):
    os.mkdir("..\\DatasetParsed")
if not os.path.exists("..\\DatasetParsed\\" + dataset_name):
    os.mkdir("..\\DatasetParsed\\" + dataset_name)

In [ ]:
counter = 0
def make_path(dataset, file_name, other = ""):
    global counter
    counter += 1
    return "..\\Plot\\" + dataset + "\\" + other + str(counter) + "-" + file_name

# Estrazione dati tabella aziende

In [ ]:
dataset_cmc_aziende = "..\\Datasets\\" + dataset_name + "\\cmc-aziende-dataset.csv"
df_a_cmc = pd.read_csv(dataset_cmc_aziende, encoding='cp1252')
df_time = df_a_cmc[["URL", "Tempo request", "Tempo estrazione Name", "Tempo estrazione CC", "Tempo estrazione marketcap", "Tempo estrazione Share price", "Tempo estrazione Revenue/Earnings", "Tempo estrazione Shares", "Tempo estrazione Categories"]] 
df_a_cmc = df_a_cmc[["URL", "ID", "Name", "Company code", "Marketcap", "Share price", "Earnings", "Revenue", "Shares", "Employees"]]
print("Numero entry tabella originale: ", len(df_a_cmc))

In [ ]:
def convert_price_type(x):
    M = 10e6
    B = 10e9
    T = 10e12
    if type(x) == str:
        x = x.replace("$", "")
        x = x.replace(",", "")
        if "M" in x:
            x = x.replace("M", "")
            x = float(x) * M
        elif "B" in x:
            x = x.replace("B", "")
            x = float(x) * B
        elif "T" in x:
            x = x.replace("T", "")
            x = float(x) * T
    return float(x)

def convert_number_type(x):
    if type(x) == str:
        x = x.replace(".", "")
    return float(x)

In [ ]:
df_a_cmc['Marketcap'] = df_a_cmc['Marketcap'].apply(convert_price_type)
df_a_cmc['Share price'] = df_a_cmc['Share price'].apply(convert_price_type)
df_a_cmc['Earnings'] = df_a_cmc['Earnings'].apply(convert_price_type)
df_a_cmc['Revenue'] = df_a_cmc['Revenue'].apply(convert_price_type)
df_a_cmc['Shares'] = df_a_cmc['Shares'].apply(convert_number_type).astype('Int64')
df_a_cmc['Employees'] = df_a_cmc['Employees'].apply(convert_number_type).astype('Int64')
df_a_cmc.to_csv("..\\DatasetParsed\\" + dataset_name + "\\" + dataset_name + "-aziende.csv", index=False)
df_a_cmc.head()

In [ ]:
info_dataset = fn.extract_data_from_df(df_a_cmc)
display(pd.DataFrame(info_dataset, index=["valore"]).T)

### Statistiche celle

In [ ]:
keys = ["celle totali", "celle nulle"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle nulle"]
]
title = "Confronto numero celle totali e vuote"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["celle totali", "celle str", "celle float", "celle int"]
values = [
    info_dataset["numero celle"], 
    info_dataset["numero celle str"],
    info_dataset["numero celle float"],
    info_dataset["numero celle int"]
]
title = "Confronto tipo celle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche righe

In [ ]:
keys = ["righe totali", "righe con celle nulle"]
values = [
    info_dataset["numero righe"], 
    info_dataset["numero righe con celle nulle"]
]
title = "Confronto numero righe totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

### Statistiche colonne

In [ ]:
keys = ["colonne totali", "colonne con celle nulle"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne con celle nulle"]
]
title = "Confronto numero colonne totali e nulle"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
keys = ["colonne totali", "colonne str", "colonne float", "colonne int"]
values = [
    info_dataset["numero colonne"], 
    info_dataset["numero colonne str"],
    info_dataset["numero colonne float"],
    info_dataset["numero colonne int"]
]
title = "Confronto tipo colonne"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim)

In [ ]:
different_value_columns = {}
for key in df_a_cmc:
    different_value_columns[key] = [len(pd.unique(df_a_cmc[key])), df_a_cmc[key].isnull().sum()]
different_value_columns = dict(sorted(different_value_columns.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys = different_value_columns.keys()
values = [item[0] for item in different_value_columns.values()]
title = "Numero di valori distinti per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

In [ ]:
keys = different_value_columns.keys()
values = [item[1] for item in different_value_columns.values()]
title = "Numero di valori nulli per colonna"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (8,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name), dim, True, info_dataset["numero righe"])

##### Pulizia righe nulle per analisi numerica

In [ ]:
df_clean = df_a_cmc[df_a_cmc['Marketcap'].isnull() == False]
df_clean = df_clean[df_clean['Share price'].isnull() == False]
df_clean = df_clean[df_clean['Earnings'].isnull() == False]
df_clean = df_clean[df_clean['Revenue'].isnull() == False]
df_clean = df_clean[df_clean['Shares'].isnull() == False]
df_clean = df_clean[df_clean['Employees'].isnull() == False]
print("Numero entry tabella dopo pulizia: ", len(df_clean))

In [ ]:
df_clean.describe()

In [ ]:
for key in ["Marketcap", "Share price", "Earnings", "Revenue", "Shares", "Employees"]:
    y = sorted(list(df_clean[key]), reverse=True)
    x = range(0,len(y))
    x_label = "Aziende"
    y_label = key
    title = "Distribuzione " + key + " per azienda"
    file_name = "".join(title.lower()).replace(" ", "_")
    dim = (15,4)
    fn.distribution_plot(x, y, x_label, y_label, title, make_path(dataset_name, file_name), dim)

# Estrazione dati tabella categorie

In [ ]:
dataset_cmc_categorie = "..\\Datasets\\" + dataset_name + "\\cmc-categorie-dataset.csv"
df_c_cmc = pd.read_csv(dataset_cmc_categorie, encoding='cp1252')
print("Numero entry tabella originale: ", len(df_c_cmc))
print("L'azienda che ha più categorie ne conta: ", max(df_c_cmc["Company ID"].value_counts()))

In [ ]:
df_c_cmc.head()

In [ ]:
info_dataset = fn.extract_data_from_df(df_c_cmc)
display(pd.DataFrame(info_dataset, index=["valore"]).T)

In [ ]:
distinct_dict = {}
category = pd.unique(df_c_cmc["Category"])
for k in category:
    distinct_dict[k] = df_c_cmc["Category"].str.count(k).sum()
distinct_dict = dict(sorted(distinct_dict.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys = distinct_dict.keys()
values = [item for item in distinct_dict.values()]
title = "Numero di aziende per categoria"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (30,5)
fn.bar_plot_ex(keys, values, title, make_path(dataset_name, file_name), dim)

##### Diagrammi di Venn

In [ ]:
distinct_dict = {}
category = pd.unique(df_c_cmc["Category"])
for k in category:
    tmp = df_c_cmc[df_c_cmc["Category"].str.contains(k)]
    distinct_dict[k] = list(tmp["Company ID"])
distinct_dict = dict(sorted(distinct_dict.items(), key=lambda item: item[1], reverse=True))

In [ ]:
for key_set in list(combinations(distinct_dict.keys(), 3)):
    subset = fn.intersection3(distinct_dict[key_set[0]], distinct_dict[key_set[1]], distinct_dict[key_set[2]])
    if subset["111"] > 4:
        fn.venn_plot(subset, key_set, path = make_path(dataset_name, "venn", "venn\\"), dim=(5,5))

##### Intersezioni

In [ ]:
num_category = 5
cond = 15
new_dict = dict((k, v) for k, v in distinct_dict.items() if len(v) > cond)
print("Numero categorie considerate: ", len(new_dict))
print("Numero di combinazioni: ", len(list(combinations(new_dict.keys(), num_category))), " (< 250 000)")

In [ ]:
# esecuzione lunga, numero di iterazioni molto elevato
# il risultato result è contenuto nel file Datasets/cmc/intersezioni.txt
i = 0
result = []
for key_set in list(combinations(new_dict.keys(), num_category)):
    d = {key: distinct_dict[key] for key in key_set} 
    dd = fn.n_intersection(d)
    print("iter: ", i)
    i += 1
    if dd[len(dd)-1][1] >= 1:
        result.append(dd)

In [ ]:
for d in result:
    values = [[]]
    data = [0]
    for i in range(0,len(d)):
        values.append(list(d[i][0]))
        data.append(d[i][1])
    example = from_memberships(values, data)
    plot(example, facecolor="#6495ED") 
    pyplot.savefig(make_path(dataset_name, "intersection"), bbox_inches='tight', transparent=True)
    pyplot.show() 

# Estrazione prestazioni

In [ ]:
# Tempo totale di esecuzione 1079.712150335312: (secondi), 1400 entry

In [ ]:
df_time.head()

In [ ]:
df_time.describe()

In [ ]:
new_df_time = df_time.describe()
del new_df_time["Tempo request"]

In [ ]:
keys = ["Tempo request", "Tempo estrazione dettagli"]
values = [df_time.describe()["Tempo request"]["mean"], new_df_time.sum(axis="columns")["mean"]]
values = np.array(values) * 1000
title = "Confronto tempo medio (ms) request vs tempo medio (ms) estrazione dettagli"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (7,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["mean"].keys())
values = list(new_df_time.loc["mean"])
values = np.array(values) * 1000
title = "Tempi medi (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = list(new_df_time.loc["max"].keys())
values = list(new_df_time.loc["max"])
values = np.array(values) * 1000
title = "Tempo massimo (ms) di estrazione dei valori (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(keys, values, title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
file_time = "..\\Datasets\\" + dataset_name + "\\tempi.txt"
file = open(file_time, 'r')
page_time = ast.literal_eval(file.read())
file.close()

In [ ]:
keys = page_time.keys()
values = [v[0] for v in page_time.values()]
values = np.array(values) * 1000
title = "Tempo (ms) per request pagina lista"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(["Pagina " + str(x) for x in keys], values.astype(int), title, make_path(dataset_name, file_name, "tempi\\"), dim, True)

In [ ]:
keys = page_time.keys()
values = [v[1] for v in page_time.values()]
values = np.array(values) * 1000
title = "Tempo (ms) di estrazione dei link delle pagine di dettaglio su ogni pagina (solo xpath)"
file_name = "".join(title.lower()).replace(" ", "_")
dim = (10,4)
fn.bar_plot(["Pagina " + str(x) for x in keys], values.astype(int), title, make_path(dataset_name, file_name, "tempi\\"), dim, True)